# Overview
This notebook illustrates the impact of the latent noise distribution on the resulting error statistics of generated samples

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext wurlitzer
#^^^ the wurlitzer extension is used to capture C/C++ output to be displayed in the notebook
#^^^ this is very useful for debugging, but it doesn't work on windows
import os
os.chdir("../")

In [ ]:
import numpy as np
from klampt.model import coordinates, trajectory
from klampt import *
from klampt.math import so3, se3
from klampt import vis

from ikflow.model_loading import get_ik_solver

In [ ]:
# PANDA
model_name="panda__full__lp191_5.25m"
ik_solver, _ = get_ik_solver(model_name)
urdf_filepath = ik_solver.robot.urdf_filepath
R = so3.from_rpy([0, np.pi/2, 0])
target_T_iktarget = (R, [-0.1, 0, 0])

# Visualizing

Per ieee: "One column width: 3.5 inches, 88.9 millimeters, or 21 picas "

Note: Don't change the color here. The standalone gui has a tool to change the appearance properties of objects in the world

When positioning the camera, try to fill the frame almost entirely with the robot.

In [ ]:
SHOW_MUG = True
SOL_MODE = "IKFLOW"
N_SOLUTIONS = 10

In [ ]:
world = ik_solver.robot._klampt_world_model
if SHOW_MUG:
    assert world.loadRigidObject("ikflow/visualization_resources/klampt_resources/objects/mug_klampt.obj")
    mug = world.rigidObject(0)
    vis.setColor(vis.getItemName(mug), 0.5, 0.5, 1.0, a=1.0)
robot = world.robot(0)

In [ ]:
# Open viz in pop up window.
vis.kill()
vis.init(["PyQt"])
vis.add("world", world)
vis.add("coordinates", coordinates.manager())
background_color = (1, 1, 1, 0.7)
vis.setBackgroundColor(background_color[0], background_color[1], background_color[2], background_color[3])
size = 5
for x0 in range(-size, size + 1):
    for y0 in range(-size, size + 1):
        vis.add(
            f"floor_{x0}_{y0}",
            trajectory.Trajectory([1, 0], [(-size, y0, 0), (size, y0, 0)]),
            color=(0.75, 0.75, 0.75, 1.0),
            width=2.0,
            hide_label=True,
            pointSize=0,
        )
        vis.add(
            f"floor_{x0}_{y0}2",
            trajectory.Trajectory([1, 0], [(x0, -size, 0), (x0, size, 0)]),
            color=(0.75, 0.75, 0.75, 1.0),
            width=2.0,
            hide_label=True,
            pointSize=0,
        )
vis.show()

In [ ]:
# Tf parameters
R = so3.from_rpy([np.pi/2, 0, -0.3])
target_T_mug = (R, [0.065, 0.005, -0.005])

# Robot parameters
alpha = 1
# robot_color = (1, 1, 1, alpha)
robot_color = (0.5, 0.5, 0.5, alpha)


def get_color(i, n):
    return robot_color

def update_sols(_end_pose):
    print(f"_end_pose = {_end_pose.tolist()}")
    if SOL_MODE == "IKFLOW":
        sols = ik_solver.solve(_end_pose, N_SOLUTIONS).cpu().numpy()
    elif SOL_MODE == "IK":
        sols = ik_solver.robot.inverse_kinematics(_end_pose, N_SOLUTIONS, debug=False)
    else:
        raise ValueError("Unknown mode")

    qs = ik_solver.robot._x_to_qs(sols)
    for i in range(N_SOLUTIONS):
        q = qs[i]
        rcolor = get_color(i, N_SOLUTIONS)
        if i == 0:
            vis.setColor(vis.getItemName(robot), rcolor[0], rcolor[1], rcolor[2], a=rcolor[3])
            robot.setConfig(q)
            continue
        vis.add(f"robot_{i}", q, color=rcolor)

def update_scene():
    target = vis.getItemConfig("target")
    world_T_target = (target[:-3], target[-3:])
    print("world_T_target =", world_T_target)
    if SHOW_MUG:
        world_T_mug = se3.mul(world_T_target, target_T_mug)
        mug.setTransform(world_T_mug[0], world_T_mug[1])
    world_T_iktarget = se3.mul(world_T_target, target_T_iktarget)
    update_sols(np.array(world_T_iktarget[1] + so3.quaternion(world_T_iktarget[0])))
    vis.update()

def current_state():
    target = vis.getItemConfig("target")
    world_T_target = (target[:-3], target[-3:])
    print("world_T_target =", world_T_target)
    print("target_T_iktarget =", target_T_iktarget)
    vp = vis.getViewport()
    print("vp = vis.getViewport()")
    print("vp.camera.rot =", vp.camera.rot)
    print("vp.camera.tgt =", vp.camera.tgt)
    print("vp.camera.dist =", vp.camera.dist)
    print("vp.clippingplanes = ", vp.clippingplanes)
    print("vis.setViewport(vp)")

### Set IK target

In [ ]:
# Update the target to a particular pose
# -- 
pose = [ 0.50, 0, 0.65,  1, 0, 0, 0]
world_T_target = ([1.0, 0, 0, 0, 1.0, 0, 0, 0, 1.0], [0.35, 0, 0.65])
vis.add("target", world_T_target, length=.25, width=1)

vis.hideLabel("target")
vis.update()

### Update scene

In [ ]:
# SOL_MODE = "IK"
SOL_MODE = "IKFLOW"
update_scene()

### Show / Hide IK target

In [ ]:
# SHOW
vis.edit("target", doedit=True)
vis.setAttribute("target", "length", .2)
vis.hideLabel("target")

In [ ]:
# HIDE
vis.edit("target", doedit=False)
vis.setAttribute("target", "length", .2)
vis.hideLabel("target")

### Printout the current vis. state

In [ ]:
current_state()